Data is strucutured as follow:
An experiment is performed on multiple cells. To increase certainty the experiment is performed multiple times (~3-4). \
During each repetition of the experience, different protocols are applied, each composed of multiple steps. Finally the results are analysed and data regarding the neuron electrical answer are generated. Those are mostly composed of spikes and inter spikes features. \
To sythetize, the data are structured as follow: Experiment/cell/repetitions/protocol/step/spike. \
As the data have to be considered at cell level we will not consider the experiment data (except for debugging)

# Librairy importation and data load

In [1]:
import os
import pandas as pd
from scipy import io


In [88]:
loaded_sp = io.loadmat('../data/01_raw/matData_2022-05-30/aCell176_1.mat')

# Functions

In [89]:
# def flatten_df(df, levels):
#     """Flatten all the columns of a datafram to facilitate data extraction
    
#     df: Dataframe to flatten
#     levels: number of wished flattening levels
#     """
#     df_flat = df.copy()
#     count = 0
#     while count<levels:
#         df_flat = df_flat.apply(lambda x: x[0])
#         count+=1
#     return df_flat

def flatten_df(df):
    """Flatten all the columns of a datafram to facilitate data extraction
    
    df: Dataframe to flatten
    """
    df_flat = df.copy()
    df_flat = df_flat.apply(lambda x: x[0][0])

    return df_flat


# Cell level data (+experiment)

In [90]:
loaded_sp.keys()
# Only aCell contains relevant information. We will not consider the other cells

dict_keys(['__header__', '__version__', '__globals__', 'aCell'])

#### First branching (but not the latest :). 
- Id simply provides the experiment ID
- Cellinfo provides the cell data
- FileInfo provides the document data (useless)
- protocol provides all theinformations regarding the stimulus and the experimental results

In [91]:
loaded_sp['aCell'][0][0].dtype

dtype([('id', 'O'), ('cellInfo', 'O'), ('protocol', 'O'), ('fileInfo', 'O')])

In [92]:
# fileInfo_temp = pd.DataFrame(loaded_sp['aCell'][0][0]['fileInfo'].ravel())
cellInfo_temp = pd.DataFrame(loaded_sp['aCell'][0][0]['cellInfo'].ravel())

                          

In [93]:
# Extract the cell info
cellInfo_flat = flatten_df(cellInfo_temp)
cellInfo_flat

id                                                   176_1
experimenter                                            OH
species                                              Mouse
channel                                                ch2
cellType                                              L5IN
path            \001_140514C01_OH\001_140514C1MMP14L5IN_OH
status                                                 [1]
dtype: object

# Extraction of data at protocol level

In [94]:
# for i in loaded_sp['aCell'][0][0]['protocol'][0]:
#     print(i[0][0][0][0])
protocol_raw = loaded_sp['aCell'][0][0]['protocol'][0][0]
protocol_raw[0][0][0][0]#['APWaveform']

'APWaveform'

#### APWaveform (Illustrative)

In [95]:
protocol_df = pd.DataFrame(protocol_raw[0][0][1][0])
protocol_df['protocol_name'] = protocol_raw[0][0][0][0]
protocol_df = protocol_df.reset_index().rename(columns={'index': 'repetition'})

protocol_df


,repetition,nTrace,ids,stim,stim_actual,stim_ids,stim_change,vHold,stim_start,stim_end,...,AP_rise_time,AP_fall_time,time_to_AP_peak,min_cur_for_discharge,AHP_duration,AHP_fall_tau,AHP_fall_A,AHP_rise_m,AHP_rise_c,protocol_name
0,0,[[6]],"[[1060, 1061, 1062, 1063, 1064, 1065]]","[[60, 100, 140, 180, 220, 260]]","[[180, 300, 420, 540, 660, 780]]","[[1, 12501, 21877, 34375]]","[[0.0, 250.0, 437.5200000000001, 687.480000000...","[[[[-69.0081558]], [[-69.09751214]], [[-69.399...",[[250]],[[437.5200000000001]],...,"[[[[]], [[]], [[0.77879995 0.85909997 0.868199...","[[[[]], [[]], [[1.28372496 1.3569214 1.341043...","[[[[]], [[]], [[ 29.02 80.08 134.38]], [[ 8....",[[[[420]]]],"[[[[]], [[]], [[42.48 46.36]], [[17.32 24.98 2...","[[[], [], [[0.98603002 1.03369811 1.12075907]]...","[[[], [], [[19.2911708 19.44998119 20.7606066...","[[[], [], [[0.39993719 0.38296232]], [[0.82177...","[[[], [], [[-146.81103099 -162.08556723]], [[-...",APWaveform
1,1,[[6]],"[[2031, 2032, 2033, 2034, 2035, 2036]]","[[60, 100, 140, 180, 220, 260]]","[[177.6, 296.0, 414.4, 532.8, 651.2, 769.6]]","[[1, 12501, 21877, 34375]]","[[0.0, 250.0, 437.5200000000001, 687.480000000...","[[[[-69.41298032]], [[-69.80458171]], [[-70.03...",[[250]],[[437.5200000000001]],...,"[[[[]], [[]], [[0.67325715 0.74319991 0.797599...","[[[[]], [[]], [[0.97225712 1.21396016 1.225586...","[[[[]], [[]], [[ 17.72 57.52 107.36 163.26]],...",[[[[414.4]]]],"[[[[]], [[]], [[33. 42.74 48.52]], [[16.46 2...","[[[], [], [[0.60947987 0.80434165 0.90259351 0...","[[[], [], [[17.57495338 20.33215333 20.0794284...","[[[], [], [[0.49138738 0.43332269 0.36965382]]...","[[[], [], [[-171.61409496 -175.39228552 -173.9...",APWaveform
2,2,[[6]],"[[3024, 3025, 3026, 3027, 3028, 3029]]","[[60, 100, 140, 180, 220, 260]]","[[180, 300, 420, 540, 660, 780]]","[[1, 12501, 21877, 34375]]","[[0.0, 250.0, 437.5200000000001, 687.480000000...","[[[[-69.5447607]], [[-70.06979898]], [[-70.194...",[[250]],[[437.5200000000001]],...,"[[[[]], [[]], [[0.69611428 0.81213329 0.844266...","[[[[]], [[]], [[1.01605968 1.2140059 1.231347...","[[[[]], [[]], [[ 17.14 59.32 111.04 165.6 ]],...",[[[[420]]]],"[[[[]], [[]], [[36.18 44.3 47.36]], [[16.58 2...","[[[], [], [[0.68684868 0.86991545 0.87491935 0...","[[[], [], [[17.86405653 21.06892433 20.5544007...","[[[], [], [[0.46137484 0.40104863 0.39585867]]...","[[[], [], [[-158.41287898 -161.3452194 -180.8...",APWaveform
3,3,[[6]],"[[4023, 4024, 4025, 4026, 4027, 4028]]","[[60, 100, 140, 180, 220, 260]]","[[180, 300, 420, 540, 660, 780]]","[[1, 12501, 21877, 34375]]","[[0.0, 250.0, 437.5200000000001, 687.480000000...","[[[[-69.45951304]], [[-69.7841089]], [[-70.168...",[[250]],[[437.5200000000001]],...,"[[[[]], [[]], [[0.68754288 0.82656 0.873200...","[[[[]], [[]], [[1.12777601 1.29030859 1.342159...","[[[[]], [[]], [[ 16.44 56.3 103.08 153.64]],...",[[[[420]]]],"[[[[]], [[]], [[33.16 38.92 42.88]], [[15.14 2...","[[[], [], [[0.85839542 1.05092076 1.08763155 1...","[[[], [], [[16.30867387 19.94087971 19.6072649...","[[[], [], [[0.48304771 0.44851787 0.41732964]]...","[[[], [], [[-159.60045926 -170.50933516 -180.5...",APWaveform


# Extraction of extracted data

In [96]:
trace_level_columns = [
    'ids',
'stim',
'stim_actual',
'vHold',
'peak_indices',
'spikecount',
'peak_time',
'peak_voltage',
'ISI_values',
'min_AHP_indices',
'min_AHP_time',
'min_AHP_voltage',
'AP_begin_voltage',
'AP_begin_time',
'AP_amplitude',
]

In [97]:
for col in trace_level_columns:
    print(col)
    print(trace_df[col].apply(lambda x: len(x)).tolist())

ids


TypeError: object of type 'numpy.uint16' has no len()

In [98]:
protocol_df.AP_begin_voltage[0]


array([[array([], shape=(1, 0), dtype=float64),
        array([], shape=(1, 0), dtype=float64),
        array([[-11.60812463, -12.58249951, -11.66499913]]),
        array([[-11.61187478,  -7.4216668 ,  -7.13875005,  -7.20749977,
                 -6.80124986,  -7.47750014]])                          ,
        array([[-8.46375196, -2.08352269, -2.33749998, -1.77208358, -1.09249999,
                -1.99875004, -3.21222226, -2.31099982]])                        ,
        array([[-4.02624969,  3.38275013,  3.476     ,  3.71000045,  2.73333325,
                 3.25458356,  2.29249995,  1.79249998,  2.41166684]])           ]],
      dtype=object)

In [87]:
trace_df = protocol_df.copy()
# trace_df[trace_level_columns] = trace_df[trace_level_columns].apply(lambda x: x.)
trace_df = trace_df.explode(trace_level_columns).explode(trace_level_columns)

In [ ]:
trace_df.head()

In [ ]:
trace_df.loc[(trace_df.repetition==0) & (trace_df.stim==125)]

In [ ]:
trace_df.loc[(trace_df.repetition==0)].vHold.tolist()

In [ ]:
# import numpy as np
# for i in trace_df.loc[(trace_df.repetition==0)].AP_begin_voltage:
#     print(np.mean(i))

In [ ]:
trace_df = protocol_df.copy()
# trace_df[trace_level_columns] = trace_df[trace_level_columns].apply(lambda x: x.)
trace_df = trace_df.explode(trace_level_columns).explode(trace_level_columns)
trace_df.head()

# Extraction of data at peak level


In [ ]:
peak_columns = [
    'peak_indices', 
'peak_time' ,
'peak_voltage', 
'min_AHP_indices', 
'min_AHP_time' ,
'min_AHP_voltage', 
'AP_begin_voltage', 
'AP_begin_time' ,
'AP_amplitude' ,
'AP_half_width' ,
'AP_width' ,
'AP_rise_time', 
'AP_fall_time' ,
'time_to_AP_peak' 
]

intra_peak_columns = [
    'ISI_values' ,
'AHP_duration' ,
'AHP_rise_m' ,
'AHP_rise_c' ,
]

In [ ]:
def peak_len_scale(value, max_peaks):
    if value.shape[0] == 0:
        val_list = []
    else:
        val_list = [t for t in value[0]]
    delta = max_peaks-len(val_list)
    val_list += delta*[0]
    return val_list

def intra_peak_len_scale(value, max_peaks):
    if value.shape[0] == 0:
        val_list = []
    else:
        val_list = [t for t in value[0]]    
    delta = max_peaks-len(val_list)
    val_list += delta*[0] 
    val_list += ['extra_inta_peak']
    
    return val_list

In [ ]:
peak_df = trace_df.copy()
peak_df.reset_index(inplace=True, drop=True)

max_peaks = max([t[0][0] for t in peak_df.spikecount])
for col in peak_columns:
    peak_df[col] = peak_df[col].apply(lambda x: peak_len_scale(x, max_peaks))

for col in intra_peak_columns:
    peak_df[col] = peak_df[col].apply(lambda x: intra_peak_len_scale(x, max_peaks-1))
peak_df = peak_df.explode(peak_columns+intra_peak_columns)



In [ ]:
peak_df.shape

# Flattening at peak level

We will now start the flattening process. At the moment, the data frame is organized in rows representing each of the peaks we observed. The new target is a row for each trace with each peak data as a column.\
For that, we will create on column per peak and feature (ex peak time of peak # 4)

In [ ]:
#Creation of a unique peak id for merging/debugging purposes
peak_df_flat = peak_df.copy()
peak_df_flat['base_temp'] = 1
peak_df_flat['peak_id_temp'] = peak_df_flat['base_temp'].cumsum()
peak_df_flat['peak_id'] = peak_df_flat['repetition'].astype('str')+ '_' +peak_df_flat['ids'].astype('str') + '_'+peak_df_flat['peak_id_temp'].astype('str')

In [ ]:
# Create a peak count column, so that we can order peaks

peak_df_flat.sort_values( 'peak_time')
grouping_df = peak_df_flat.sort_values( 'peak_time')[['repetition', 'peak_id', 'ids', 'base_temp']]
grouping_df['peak_rank'] = grouping_df.groupby(['repetition', 'ids']).cumsum()


In [ ]:
peak_df_flat = peak_df_flat.merge(grouping_df[['peak_rank', 'peak_id']] ,how='left', on='peak_id')


In [ ]:
peak_df_flat.head()

In [ ]:
peak_df_flat = peak_df_flat.pivot_table(index=['repetition', 'ids'], columns='peak_rank', values= intra_peak_columns+peak_columns )

peak_df_flat.columns = [ f'{column[0]}_{column[1]}' for column in peak_df_flat.columns]
peak_df_flat.reset_index(inplace=True)

peak_df_flat.head()

In [ ]:
peak_df_flat.shape

# Flatten at trace level

In [ ]:
peak_df_cols = peak_df_flat.columns.tolist()

peak_feat = [f'{col}_trace' for col in peak_df_cols if col not in ['repetition', 'ids']]
trace_level_columns_pivot = [f'{col}_trace' for col in trace_level_columns if col not in ['repetition', 'ids']]




In [ ]:
# Integrate back trace data

trace_keys = ['repetition', 'ids']

trace_df_flat = peak_df_flat.merge(trace_df[trace_level_columns+['repetition']], on=trace_keys, how='left')

trace_df_flat.columns = [f'{col}_trace' for col in trace_df_flat.columns]
trace_df_flat['protocol_name'] = protocol_raw[0][0][0][0]

# peak_df_flat = peak_df_flat.pivot_table(index=['repetition', 'ids'], columns='peak_rank', values= intra_peak_columns+peak_columns )

trace_df_flat = trace_df_flat.pivot_table(index= 'protocol_name', columns='repetition_trace', values= peak_feat + trace_level_columns_pivot )

trace_df_flat.columns = [ f'{column[0]}_{column[1]}' for column in trace_df_flat.columns]
trace_df_flat.reset_index(inplace=True)

trace_df_flat.head()
